In [6]:
import degirum as dg, degirum_tools
#import os
from pathlib import Path
import gen_test_image_util as imgutil
#import importlib
import os
import time
import cv2
#import threading
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

#importlib.reload(imgutil)

In [7]:
#hw_location = "@cloud"
hw_location = "@local"
orca1_zoo_url = "https://cs.degirum.com/degirum/public"
orca_zoo_url = "https://cs.degirum.com/degirum/orca_models"
#model_zoo_url = "https://cs.degirum.com/degirum/ultralytics_v6"
device_family = "ORCA1"  # models of which device family to use
exclude_preprocessing = True
use_jpeg = True

image_dir = Path('/home/gotom/data_dir/split_pic')

num_of_pics = 200
#max_count = 3000

In [ ]:
#*******************************************************************************************************
# multi_model_parallel_test()
#*******************************************************************************************************
def multi_models_parallel_test(model_names, model_instances, image_file,\
                                exclude_preprocessing, iterations, batch):
    ret = {}
    data = []   
    nmodels = len(model_instances)
    for mi, model_name in enumerate(model_names):
        ret[model_name] = {}
        frame = image_file
        if exclude_preprocessing:
            frame = model_instances[mi]._preprocessor.forward(frame)[0]
        data.append(frame)
    # define source of frames
    def source(mi):
        for fi in range(iterations):
            yield data[mi]  
    barr = threading.Barrier(nmodels)

    def run_one_model(mi):
        with model_instances[mi] as model:
            model.eager_batch_size = batch
            model.frame_queue_depth = batch
            model(data[mi])  # run model once to warm up the system
            model.reset_time_stats()
            barr.wait()
            t = dgtools.Timer()          
            for res in model.predict_batch(source(mi,)):
                pass
            ret[model_names[mi]] = {
                    "elapsed": t(),
                    "time_stats": model.time_stats(),
                }
    threads = [
            threading.Thread(target=run_one_model, args=(mi,)) for mi in range(nmodels)
        ]
    [thread.start() for thread in threads]
    [thread.join() for thread in threads]

    return ret

In [ ]:
#*******************************************************************************************************
# multi_model_parallel_test2()
#*******************************************************************************************************
def multi_models_parallel_test(model_names, model_instances, image_file,\
                                exclude_preprocessing, iterations, batch):
    ret = {}
    data = []   
    nmodels = len(model_instances)
    for mi, model_name in enumerate(model_names):
        ret[model_name] = {}
        frame = image_file
        if exclude_preprocessing:
            frame = model_instances[mi]._preprocessor.forward(frame)[0]
        data.append(frame)
    # define source of frames
    # def source(mi):
    #     for fi in range(iterations):
    #         yield data[mi]  
    # barr = threading.Barrier(nmodels)

    # def run_one_model(mi):
    #     with model_instances[mi] as model:
    #         model.eager_batch_size = batch
    #         model.frame_queue_depth = batch
    #         model(data[mi])  # run model once to warm up the system
    #         model.reset_time_stats()
    #         barr.wait()
            t = dgtools.Timer()          
    #         for res in model.predict_batch(source(mi,)):
    #             pass
    #         ret[model_names[mi]] = {
    #                 "elapsed": t(),
    #                 "time_stats": model.time_stats(),
    #             }
    # threads = [
    #         threading.Thread(target=run_one_model, args=(mi,)) for mi in range(nmodels)
    #     ]
    # [thread.start() for thread in threads]
    # [thread.join() for thread in threads]

    return ret

In [8]:
def partial_transaction(model_instance, image_chunk_pathL, iterations):
    
    count = 0
    #print(f'start_index:{start_index}')
    try:
        for res in model_instance.predict_batch(imgutil.pics_transaction_gen(model_instance, image_chunk_pathL[:iterations], exclude_preprocessing, iterations)):
            count += 1
            #pass
    except Exception as e:
        print(e)
    
    #print(f'count:{count}')
    return count

In [9]:
image_size = [224, 224, 200]
      
mobile_net_v1_durationD = {}
mobile_net_v2_durationD = {}

batch_size = 16


In [10]:
imgutil.remove_image_chunk(image_dir, 3000)
image_chunkL = imgutil.create_test_image_chunk('maami_1.jpg', image_size)
image_chunk_pathL = imgutil.save_image_chunk(image_dir, image_chunkL)
    
orca_zoo = dg.connect(hw_location, orca_zoo_url, degirum_tools.get_token()) # connect to AI inference engine
orca1_zoo = dg.connect(hw_location, orca1_zoo_url, degirum_tools.get_token()) # connect to AI inference engine

#model_orca = 'mobilenet_v1_imagenet--224x224_quant_n2x_orca_1'
#model_orca1 = 'mobilenet_v1_imagenet--224x224_quant_n2x_orca1_1'   
model_orca = 'mobilenet_v2_imagenet--224x224_quant_n2x_orca_1'
model_orca1 = 'mobilenet_v2_imagenet--224x224_quant_n2x_orca1_1'   
exclude_preprocessing = True

durationL = []
          
image = cv2.imread(image_chunk_pathL[0], cv2.IMREAD_COLOR)
h,w = image.shape[:2]
image_size[0] = h
image_size[1] = w

instance_orca = imgutil.create_model_instances(model_orca, orca_zoo, batch_size, use_jpeg)
instance_orca1 = imgutil.create_model_instances(model_orca1, orca1_zoo, batch_size, use_jpeg)
#instance_orca = instance_orca[0]
#instance_orca1 = instance_orca1[0]

instance_orca(image_chunk_pathL[0]) #dummy
instance_orca1(image_chunk_pathL[100]) #dummy

instance_orca.reset_time_stats()
instance_orca1.reset_time_stats()

count = 0
countL = []
resultL = []
        
#single device orca
t = degirum_tools.Timer()
count = partial_transaction(instance_orca, image_chunk_pathL, 0, 200)
orca_elapsed_time = round(1e3*t(),1)
#print(f'count = {count}')  
#print(f'mobile_net_v1_duration (ORCA single device): {elapsed_time}msec')

#single device orca1
t = degirum_tools.Timer()
count = partial_transaction(instance_orca1, image_chunk_pathL, 0, 200)
orca1_elapsed_time = round(1e3*t(),1)
#print(f'count = {count}') 
#print(f'mobile_net_v1_duration (ORCA1 single device): {elapsed_time}msec')

#two devices orca + orca1
t = degirum_tools.Timer()
with ThreadPoolExecutor(max_workers = 2) as exctor:
    count_orca = exctor.submit(partial_transaction, instance_orca, image_chunk_pathL, 0, 100)
    count_orca1 = exctor.submit(partial_transaction, instance_orca1, image_chunk_pathL, 100, 100) 
    
    #print(f'countL:{countL}')
    resultL = [count_orca.result(), count_orca1.result()]   
    #print(f'resultL = {resultL}')              
    two_orca_elapsed_time = round(1e3*t(),1)  
    #print(f'count:{sum(resultL)}')
    if not(sum(resultL) == num_of_pics): 
        exit()
    
    #height_width = str(h) + 'x' + str(w)
    #mobile_net_v1_durationD[height_width] = elapsed_time
    #mobile_net_v2_durationD[height_width] = elapsed_time
            
#print(f'mobile_net_v1_duration (ORCA + ORCA1): {elapsed_time}msec')


In [11]:
#print results    
CW = (50, 20, 30)  # column widths
header = f"| {'Model name':{CW[0]}}| {'Device':{CW[1]}}| {'200 Images Inference Time[ms]':{CW[2]}}|"

print('200 images transaction demo (Single device, Two devices)\n')
print(f"hw_location:  {hw_location}")
print(f"Batch Size:   16")
print(f'Image Size:   224 x 224')
print(f"{'-'*len(header)}")
print(header)
print(f"{'-'*len(header)}")

print(f"| {model_orca:{CW[0]}}|" + f"{' ORCA':{CW[1]}} |" + f" {orca_elapsed_time:{CW[2]}}|")
print(f"| {model_orca1:{CW[0]}}|" + f"{' ORCA1':{CW[1]}} |" + f" {orca1_elapsed_time:{CW[2]}}|")
print(f"| {model_orca1:{CW[0]}}|" + f"{' ORCA + ORCA1':{CW[1]}} |" + f" {two_orca_elapsed_time:{CW[2]}}|")  
    
print(f"{'-'*len(header)}") 

200 images transaction demo (Single device, Two devices)

hw_location:  @local
Batch Size:   16
Image Size:   224 x 224
-----------------------------------------------------------------------------------------------------------
| Model name                                        | Device              | 200 Images Inference Time[ms] |
-----------------------------------------------------------------------------------------------------------
| mobilenet_v2_imagenet--224x224_quant_n2x_orca_1   | ORCA                |                          947.2|
| mobilenet_v2_imagenet--224x224_quant_n2x_orca1_1  | ORCA1               |                          614.6|
| mobilenet_v2_imagenet--224x224_quant_n2x_orca1_1  | ORCA + ORCA1        |                          485.5|
-----------------------------------------------------------------------------------------------------------
